In [1]:
import sys
import os
import glob
import re 
import pandas as pd
import numpy as np
from lxml import etree
import matplotlib.pyplot as plt
import requests
import time
from collections import Counter


In [2]:
spanish_metadata_df = pd.read_csv("./../data/spanish_metadata_translations.tsv", sep="\t", index_col=0)

In [3]:
spanish_metadata_df = spanish_metadata_df.loc[spanish_metadata_df["language"] == "es"]

In [4]:
spanish_metadata_df

,language,link,name,code,about,copyright,years_about,year_about,years_copyright,year_copyright,number_verses,number_chapters,number_books,year
87,es,/versions/La-Biblia-de-las-Américas-LBLA/#book...,La Biblia de las Américas (LBLA),LBLA,\nVersion Information\n\nLa Biblia de las Amér...,\nCopyright Information\n\nPermission To Quote...,"['1569', '1986']",1569.0,"['1986', '1995', '1997', '2279', '3055', '1986...",1986.0,31103.0,1189.0,66.0,1986.0
88,es,/versions/Jubilee-Bible-2000-Spanish-JBS/#book...,Biblia del Jubileo (JBS),JBS,\nVersion Information\n\nEn la Biblia Jubileo...,\nCopyright Information\n\nLas Sagradas Escrit...,['2000'],2000.0,"['2000', '2000', '2001', '2010', '2014', '2017...",2000.0,31102.0,1189.0,66.0,2000.0
89,es,/versions/Dios-Habla-Hoy-DHH-Biblia/#booklist,Dios Habla Hoy (DHH),DHH,\nVersion Information\n\nVersion information i...,\nCopyright Information\n\nScriptures marked a...,[],NaN,"['1966', '1970', '1979', '1983', '1996', '1989']",1966.0,30753.0,1189.0,66.0,1966.0
90,es,/versions/Nueva-Biblia-de-las-Américas-NBLA/#b...,Nueva Biblia de las Américas (NBLA),NBLA,\nVersion Information\n\nNueva Biblia de las ...,\nCopyright Information\n\nNueva Biblia de las...,[],NaN,"['2005', '1986', '1995', '1997']",2005.0,31103.0,1189.0,66.0,2005.0
91,es,/versions/Nueva-Biblia-Viva-NBV/#booklist,Nueva Biblia Viva (NBV),NBV,\nVersion Information\n\n No ...,\nCopyright Information\n\nNew Living Bible (N...,[],NaN,"['2006', '2008']",2006.0,29102.0,1189.0,66.0,2006.0
92,es,/versions/Nueva-Traduccion-Viviente-Biblia-NTV...,Nueva Traducción Viviente (NTV),NTV,"\nVersion Information\n\nLa Santa Biblia, Nuev...",\nCopyright Information\n\nVisite Tyndale en I...,[],NaN,"['2010', '2010', '2010', '2010', '8300', '5023']",2010.0,30931.0,1189.0,66.0,2010.0
93,es,/versions/Nueva-Version-Internacional-Castilia...,Nueva Versión Internacional (Castilian) (CST),CST,\nVersion Information\n\n No ...,"\nCopyright Information\n\nSanta Biblia, NUEVA...",[],NaN,"['1999', '2005', '2017']",1999.0,30955.0,1189.0,66.0,1999.0
94,es,/versions/Nueva-Version-Internacional-Biblia-N...,Nueva Versión Internacional (NVI),NVI,"\nVersion Information\n\nIn 1979, The Internat...","\nCopyright Information\n\nSanta Biblia, NUEVA...","['1979', '2015']",1979.0,"['1999', '2015']",1999.0,30969.0,1189.0,66.0,1999.0
95,es,/versions/Palabra-de-Dios-para-Todos-Biblia-PD...,Palabra de Dios para Todos (PDT),PDT,\nVersion Information\n\nEsta versión de la Bi...,\nCopyright Information\n\nLa Biblia : La Pala...,[],NaN,"['2005', '2008', '2012', '2015', '2005', '2008...",2005.0,31083.0,1189.0,66.0,2005.0
96,es,/versions/Biblia-La-Palabra-España-BLP/#booklist,La Palabra (España) (BLP),BLP,\nVersion Information\n\nThe Bible Society of ...,"\nCopyright Information\n\nLa Palabra, (versió...",[],NaN,['2010'],2010.0,31163.0,1189.0,66.0,2010.0


In [5]:
metadata_df = pd.read_csv("./../data/multilingual_metadata_translations.tsv", sep="\t", index_col=0)

In [6]:
metadata_df.shape

(236, 100)

In [7]:
metadata_df

,file_name,file,language,title_name,to_be_analyzed?,title_name_file,year,analysis,years_title_name_file,year_title_name_file,...,bsname_71,bsname_72,bsname_73,bsname_74,bsname_75,bsname_76,bsname_77,bsname_78,bsname_79,bsname_80
232,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALSSHQ1994EB_(ALBANIAN BIBLE TOSK),1,Albanian Bible Tosk,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SF_2009-01-20_ALB_ALB1_(ALBANIAN VERSION),./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALB1_(ALBANIAN VERSION),1,Albanian Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARANAV_(NEW ARABIC VERSION (KETAB EL HAYAT)),1,New Arabic Version (Ketab El Hayat),NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SF_2009-01-20_ARA_ARASVD_(SMITH VAN DYKE ARABI...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARASVD_(SMITH VAN DYKE ARABIC BIBLE),1,Smith Van Dyke Arabic Bible,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHE...,./../data/bibles/transformed_xml_with_year\SF_...,AZB,AZERI_(DESCRIPTION_EN=SOUTHERN AZERI TRANSLATION),1,Description_en=Southern Azeri Translation,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,SF_2009-01-20_VIE_VIE_(VIETNAMESE VERSION),./../data/bibles/transformed_xml_with_year\SF_...,VIE,VIE_(VIETNAMESE VERSION),1,Vietnamese Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,SF_2014-05-12_WIU_WITNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,WIU,WITNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF_2009-01-22_XKL_KLINGON_(KLINGON VERSION),./../data/bibles/transformed_xml_with_year\SF_...,XKL,KLINGON_(KLINGON VERSION),1,Klingon Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,SF_2014-05-12_YBY_YAWNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,YBY,YAWNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
metadata_df.loc[metadata_df["analysis"] == "yes"]

,file_name,file,language,title_name,to_be_analyzed?,title_name_file,year,analysis,years_title_name_file,year_title_name_file,...,bsname_71,bsname_72,bsname_73,bsname_74,bsname_75,bsname_76,bsname_77,bsname_78,bsname_79,bsname_80
232,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALSSHQ1994EB_(ALBANIAN BIBLE TOSK),1,Albanian Bible Tosk,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SF_2009-01-20_ALB_ALB1_(ALBANIAN VERSION),./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALB1_(ALBANIAN VERSION),1,Albanian Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARANAV_(NEW ARABIC VERSION (KETAB EL HAYAT)),1,New Arabic Version (Ketab El Hayat),NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SF_2009-01-20_ARA_ARASVD_(SMITH VAN DYKE ARABI...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARASVD_(SMITH VAN DYKE ARABIC BIBLE),1,Smith Van Dyke Arabic Bible,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHE...,./../data/bibles/transformed_xml_with_year\SF_...,AZB,AZERI_(DESCRIPTION_EN=SOUTHERN AZERI TRANSLATION),1,Description_en=Southern Azeri Translation,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,SF_2009-01-20_VIE_VIE_(VIETNAMESE VERSION),./../data/bibles/transformed_xml_with_year\SF_...,VIE,VIE_(VIETNAMESE VERSION),1,Vietnamese Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,SF_2014-05-12_WIU_WITNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,WIU,WITNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF_2009-01-22_XKL_KLINGON_(KLINGON VERSION),./../data/bibles/transformed_xml_with_year\SF_...,XKL,KLINGON_(KLINGON VERSION),1,Klingon Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,SF_2014-05-12_YBY_YAWNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,YBY,YAWNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
metadata_df = metadata_df.loc[metadata_df["analysis"] == "yes"].copy()

In [10]:
metadata_df

,file_name,file,language,title_name,to_be_analyzed?,title_name_file,year,analysis,years_title_name_file,year_title_name_file,...,bsname_71,bsname_72,bsname_73,bsname_74,bsname_75,bsname_76,bsname_77,bsname_78,bsname_79,bsname_80
232,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALSSHQ1994EB_(ALBANIAN BIBLE TOSK),1,Albanian Bible Tosk,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SF_2009-01-20_ALB_ALB1_(ALBANIAN VERSION),./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALB1_(ALBANIAN VERSION),1,Albanian Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARANAV_(NEW ARABIC VERSION (KETAB EL HAYAT)),1,New Arabic Version (Ketab El Hayat),NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SF_2009-01-20_ARA_ARASVD_(SMITH VAN DYKE ARABI...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARASVD_(SMITH VAN DYKE ARABIC BIBLE),1,Smith Van Dyke Arabic Bible,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHE...,./../data/bibles/transformed_xml_with_year\SF_...,AZB,AZERI_(DESCRIPTION_EN=SOUTHERN AZERI TRANSLATION),1,Description_en=Southern Azeri Translation,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,SF_2009-01-20_VIE_VIE_(VIETNAMESE VERSION),./../data/bibles/transformed_xml_with_year\SF_...,VIE,VIE_(VIETNAMESE VERSION),1,Vietnamese Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,SF_2014-05-12_WIU_WITNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,WIU,WITNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF_2009-01-22_XKL_KLINGON_(KLINGON VERSION),./../data/bibles/transformed_xml_with_year\SF_...,XKL,KLINGON_(KLINGON VERSION),1,Klingon Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,SF_2014-05-12_YBY_YAWNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,YBY,YAWNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
metadata_df["source"] = "zefania"

In [12]:
metadata_df.columns.tolist()

['file_name',
 'file',
 'language',
 'title_name',
 'to_be_analyzed?',
 'title_name_file',
 'year',
 'analysis',
 'years_title_name_file',
 'year_title_name_file',
 'years_description_name',
 'year_description_name',
 'creator_name',
 'description_name',
 'publisher_name',
 'contributors_name',
 'source_name',
 'number_verses',
 'number_chapters',
 'number_books',
 'bsname_1',
 'bsname_2',
 'bsname_3',
 'bsname_4',
 'bsname_5',
 'bsname_6',
 'bsname_7',
 'bsname_8',
 'bsname_9',
 'bsname_10',
 'bsname_11',
 'bsname_12',
 'bsname_13',
 'bsname_14',
 'bsname_15',
 'bsname_16',
 'bsname_17',
 'bsname_18',
 'bsname_19',
 'bsname_20',
 'bsname_21',
 'bsname_22',
 'bsname_23',
 'bsname_24',
 'bsname_25',
 'bsname_26',
 'bsname_27',
 'bsname_28',
 'bsname_29',
 'bsname_30',
 'bsname_31',
 'bsname_32',
 'bsname_33',
 'bsname_34',
 'bsname_35',
 'bsname_36',
 'bsname_37',
 'bsname_38',
 'bsname_39',
 'bsname_40',
 'bsname_41',
 'bsname_42',
 'bsname_43',
 'bsname_44',
 'bsname_45',
 'bsname_46'

In [13]:
metadata_df

,file_name,file,language,title_name,to_be_analyzed?,title_name_file,year,analysis,years_title_name_file,year_title_name_file,...,bsname_72,bsname_73,bsname_74,bsname_75,bsname_76,bsname_77,bsname_78,bsname_79,bsname_80,source
232,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALSSHQ1994EB_(ALBANIAN BIBLE TOSK),1,Albanian Bible Tosk,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
7,SF_2009-01-20_ALB_ALB1_(ALBANIAN VERSION),./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALB1_(ALBANIAN VERSION),1,Albanian Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
161,SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARANAV_(NEW ARABIC VERSION (KETAB EL HAYAT)),1,New Arabic Version (Ketab El Hayat),NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
8,SF_2009-01-20_ARA_ARASVD_(SMITH VAN DYKE ARABI...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARASVD_(SMITH VAN DYKE ARABIC BIBLE),1,Smith Van Dyke Arabic Bible,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
162,SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHE...,./../data/bibles/transformed_xml_with_year\SF_...,AZB,AZERI_(DESCRIPTION_EN=SOUTHERN AZERI TRANSLATION),1,Description_en=Southern Azeri Translation,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,SF_2009-01-20_VIE_VIE_(VIETNAMESE VERSION),./../data/bibles/transformed_xml_with_year\SF_...,VIE,VIE_(VIETNAMESE VERSION),1,Vietnamese Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
199,SF_2014-05-12_WIU_WITNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,WIU,WITNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
123,SF_2009-01-22_XKL_KLINGON_(KLINGON VERSION),./../data/bibles/transformed_xml_with_year\SF_...,XKL,KLINGON_(KLINGON VERSION),1,Klingon Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
200,SF_2014-05-12_YBY_YAWNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,YBY,YAWNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania


In [14]:
filetered_metadata_df = metadata_df.loc[(metadata_df["language"] != "SPA") | (metadata_df["title_name_file"] != "Spanish Reina-Valera")]

In [15]:
filetered_metadata_df

,file_name,file,language,title_name,to_be_analyzed?,title_name_file,year,analysis,years_title_name_file,year_title_name_file,...,bsname_72,bsname_73,bsname_74,bsname_75,bsname_76,bsname_77,bsname_78,bsname_79,bsname_80,source
232,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALSSHQ1994EB_(ALBANIAN BIBLE TOSK),1,Albanian Bible Tosk,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
7,SF_2009-01-20_ALB_ALB1_(ALBANIAN VERSION),./../data/bibles/transformed_xml_with_year\SF_...,ALB,ALB1_(ALBANIAN VERSION),1,Albanian Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
161,SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARANAV_(NEW ARABIC VERSION (KETAB EL HAYAT)),1,New Arabic Version (Ketab El Hayat),NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
8,SF_2009-01-20_ARA_ARASVD_(SMITH VAN DYKE ARABI...,./../data/bibles/transformed_xml_with_year\SF_...,ARA,ARASVD_(SMITH VAN DYKE ARABIC BIBLE),1,Smith Van Dyke Arabic Bible,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
162,SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHE...,./../data/bibles/transformed_xml_with_year\SF_...,AZB,AZERI_(DESCRIPTION_EN=SOUTHERN AZERI TRANSLATION),1,Description_en=Southern Azeri Translation,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,SF_2009-01-20_VIE_VIE_(VIETNAMESE VERSION),./../data/bibles/transformed_xml_with_year\SF_...,VIE,VIE_(VIETNAMESE VERSION),1,Vietnamese Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
199,SF_2014-05-12_WIU_WITNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,WIU,WITNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
123,SF_2009-01-22_XKL_KLINGON_(KLINGON VERSION),./../data/bibles/transformed_xml_with_year\SF_...,XKL,KLINGON_(KLINGON VERSION),1,Klingon Version,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania
200,SF_2014-05-12_YBY_YAWNT_(THE NEW TESTAMENT),./../data/bibles/transformed_xml_with_year\SF_...,YBY,YAWNT_(THE NEW TESTAMENT),1,The New Testament,NaN,yes,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zefania


In [16]:
metadata_df.rename(columns = {"file" : "path", "title_name_file" : "title"}, inplace=True)

In [17]:
spanish_metadata_df["language"].replace("es", "SPA", inplace=True)

In [18]:
spanish_metadata_df["source"] = "biblegateway"

In [19]:
spanish_metadata_df

,language,link,name,code,about,copyright,years_about,year_about,years_copyright,year_copyright,number_verses,number_chapters,number_books,year,source
87,SPA,/versions/La-Biblia-de-las-Américas-LBLA/#book...,La Biblia de las Américas (LBLA),LBLA,\nVersion Information\n\nLa Biblia de las Amér...,\nCopyright Information\n\nPermission To Quote...,"['1569', '1986']",1569.0,"['1986', '1995', '1997', '2279', '3055', '1986...",1986.0,31103.0,1189.0,66.0,1986.0,biblegateway
88,SPA,/versions/Jubilee-Bible-2000-Spanish-JBS/#book...,Biblia del Jubileo (JBS),JBS,\nVersion Information\n\nEn la Biblia Jubileo...,\nCopyright Information\n\nLas Sagradas Escrit...,['2000'],2000.0,"['2000', '2000', '2001', '2010', '2014', '2017...",2000.0,31102.0,1189.0,66.0,2000.0,biblegateway
89,SPA,/versions/Dios-Habla-Hoy-DHH-Biblia/#booklist,Dios Habla Hoy (DHH),DHH,\nVersion Information\n\nVersion information i...,\nCopyright Information\n\nScriptures marked a...,[],NaN,"['1966', '1970', '1979', '1983', '1996', '1989']",1966.0,30753.0,1189.0,66.0,1966.0,biblegateway
90,SPA,/versions/Nueva-Biblia-de-las-Américas-NBLA/#b...,Nueva Biblia de las Américas (NBLA),NBLA,\nVersion Information\n\nNueva Biblia de las ...,\nCopyright Information\n\nNueva Biblia de las...,[],NaN,"['2005', '1986', '1995', '1997']",2005.0,31103.0,1189.0,66.0,2005.0,biblegateway
91,SPA,/versions/Nueva-Biblia-Viva-NBV/#booklist,Nueva Biblia Viva (NBV),NBV,\nVersion Information\n\n No ...,\nCopyright Information\n\nNew Living Bible (N...,[],NaN,"['2006', '2008']",2006.0,29102.0,1189.0,66.0,2006.0,biblegateway
92,SPA,/versions/Nueva-Traduccion-Viviente-Biblia-NTV...,Nueva Traducción Viviente (NTV),NTV,"\nVersion Information\n\nLa Santa Biblia, Nuev...",\nCopyright Information\n\nVisite Tyndale en I...,[],NaN,"['2010', '2010', '2010', '2010', '8300', '5023']",2010.0,30931.0,1189.0,66.0,2010.0,biblegateway
93,SPA,/versions/Nueva-Version-Internacional-Castilia...,Nueva Versión Internacional (Castilian) (CST),CST,\nVersion Information\n\n No ...,"\nCopyright Information\n\nSanta Biblia, NUEVA...",[],NaN,"['1999', '2005', '2017']",1999.0,30955.0,1189.0,66.0,1999.0,biblegateway
94,SPA,/versions/Nueva-Version-Internacional-Biblia-N...,Nueva Versión Internacional (NVI),NVI,"\nVersion Information\n\nIn 1979, The Internat...","\nCopyright Information\n\nSanta Biblia, NUEVA...","['1979', '2015']",1979.0,"['1999', '2015']",1999.0,30969.0,1189.0,66.0,1999.0,biblegateway
95,SPA,/versions/Palabra-de-Dios-para-Todos-Biblia-PD...,Palabra de Dios para Todos (PDT),PDT,\nVersion Information\n\nEsta versión de la Bi...,\nCopyright Information\n\nLa Biblia : La Pala...,[],NaN,"['2005', '2008', '2012', '2015', '2005', '2008...",2005.0,31083.0,1189.0,66.0,2005.0,biblegateway
96,SPA,/versions/Biblia-La-Palabra-España-BLP/#booklist,La Palabra (España) (BLP),BLP,\nVersion Information\n\nThe Bible Society of ...,"\nCopyright Information\n\nLa Palabra, (versió...",[],NaN,['2010'],2010.0,31163.0,1189.0,66.0,2010.0,biblegateway


In [20]:
columns = ["path", "language", "title",'year', 'number_verses', 'number_chapters', 'number_books', "source"]

In [21]:
spanish_metadata_df.rename(columns = {"name" : "title", "link" : "path"}, inplace=True)

In [22]:

joined_metadata_df = pd.concat([metadata_df[columns], spanish_metadata_df[columns]]) 

In [23]:
joined_metadata_df

,path,language,title,year,number_verses,number_chapters,number_books,source
232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,NaN,31102.0,1189.0,66.0,zefania
7,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Version,NaN,31102.0,1189.0,66.0,zefania
161,./../data/bibles/transformed_xml_with_year\SF_...,ARA,New Arabic Version (Ketab El Hayat),NaN,31101.0,1189.0,66.0,zefania
8,./../data/bibles/transformed_xml_with_year\SF_...,ARA,Smith Van Dyke Arabic Bible,NaN,31102.0,1189.0,66.0,zefania
162,./../data/bibles/transformed_xml_with_year\SF_...,AZB,Description_en=Southern Azeri Translation,NaN,31102.0,1189.0,66.0,zefania
...,...,...,...,...,...,...,...,...
101,/versions/Reina-Valera-Revisada-RVR/#booklist,SPA,Reina Valera Revisada (RVR1977),1977,31103.0,1189.0,66.0,biblegateway
102,/versions/Reina-Valera-1995-RVR1995-Biblia/#bo...,SPA,Reina-Valera 1995 (RVR1995),1995,31104.0,1189.0,66.0,biblegateway
103,/versions/Reina-Valera-Antigua-RVA-Biblia/#boo...,SPA,Reina-Valera Antigua (RVA),1569,31101.0,1189.0,66.0,biblegateway
104,/versions/Spanish-Blue-Red-Gold-Letter-Edition...,SPA,Spanish Blue Red and Gold Letter Edition (SRV-...,2014,31103.0,1189.0,66.0,biblegateway


In [24]:
joined_metadata_df.year.replace("?", np.NaN, inplace=True)

In [25]:
joined_metadata_df.year = joined_metadata_df.year.fillna(0).astype(int)

In [26]:
joined_metadata_df.number_verses = joined_metadata_df.number_verses.astype(int)
joined_metadata_df.number_chapters = joined_metadata_df.number_chapters.astype(int)
joined_metadata_df.number_books = joined_metadata_df.number_books.astype(int)


In [27]:
joined_metadata_df

,path,language,title,year,number_verses,number_chapters,number_books,source
232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania
7,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Version,0,31102,1189,66,zefania
161,./../data/bibles/transformed_xml_with_year\SF_...,ARA,New Arabic Version (Ketab El Hayat),0,31101,1189,66,zefania
8,./../data/bibles/transformed_xml_with_year\SF_...,ARA,Smith Van Dyke Arabic Bible,0,31102,1189,66,zefania
162,./../data/bibles/transformed_xml_with_year\SF_...,AZB,Description_en=Southern Azeri Translation,0,31102,1189,66,zefania
...,...,...,...,...,...,...,...,...
101,/versions/Reina-Valera-Revisada-RVR/#booklist,SPA,Reina Valera Revisada (RVR1977),1977,31103,1189,66,biblegateway
102,/versions/Reina-Valera-1995-RVR1995-Biblia/#bo...,SPA,Reina-Valera 1995 (RVR1995),1995,31104,1189,66,biblegateway
103,/versions/Reina-Valera-Antigua-RVA-Biblia/#boo...,SPA,Reina-Valera Antigua (RVA),1569,31101,1189,66,biblegateway
104,/versions/Spanish-Blue-Red-Gold-Letter-Edition...,SPA,Spanish Blue Red and Gold Letter Edition (SRV-...,2014,31103,1189,66,biblegateway


In [28]:
joined_metadata_df.loc[(joined_metadata_df["source"] == "zefania") & (joined_metadata_df["language"] == "SPA") ]

,path,language,title,year,number_verses,number_chapters,number_books,source
106,./../data/bibles/transformed_xml_with_year\SF_...,SPA,Las Sagradas Escrituras,0,31102,1189,66,zefania
105,./../data/bibles/transformed_xml_with_year\SF_...,SPA,Reina Valera 1989,0,31127,1189,66,zefania
122,./../data/bibles/transformed_xml_with_year\SF_...,SPA,Spanish Reina-Valera,0,31102,1189,66,zefania


In [29]:
joined_metadata_df.drop(index=106, inplace=True)

In [30]:
joined_metadata_df.drop(index=122, inplace=True)

In [31]:
joined_metadata_df.loc[joined_metadata_df["language"] == "SPA"]

,path,language,title,year,number_verses,number_chapters,number_books,source
105,./../data/bibles/transformed_xml_with_year\SF_...,SPA,Reina Valera 1989,0,31127,1189,66,zefania
87,/versions/La-Biblia-de-las-Américas-LBLA/#book...,SPA,La Biblia de las Américas (LBLA),1986,31103,1189,66,biblegateway
88,/versions/Jubilee-Bible-2000-Spanish-JBS/#book...,SPA,Biblia del Jubileo (JBS),2000,31102,1189,66,biblegateway
89,/versions/Dios-Habla-Hoy-DHH-Biblia/#booklist,SPA,Dios Habla Hoy (DHH),1966,30753,1189,66,biblegateway
90,/versions/Nueva-Biblia-de-las-Américas-NBLA/#b...,SPA,Nueva Biblia de las Américas (NBLA),2005,31103,1189,66,biblegateway
91,/versions/Nueva-Biblia-Viva-NBV/#booklist,SPA,Nueva Biblia Viva (NBV),2006,29102,1189,66,biblegateway
92,/versions/Nueva-Traduccion-Viviente-Biblia-NTV...,SPA,Nueva Traducción Viviente (NTV),2010,30931,1189,66,biblegateway
93,/versions/Nueva-Version-Internacional-Castilia...,SPA,Nueva Versión Internacional (Castilian) (CST),1999,30955,1189,66,biblegateway
94,/versions/Nueva-Version-Internacional-Biblia-N...,SPA,Nueva Versión Internacional (NVI),1999,30969,1189,66,biblegateway
95,/versions/Palabra-de-Dios-para-Todos-Biblia-PD...,SPA,Palabra de Dios para Todos (PDT),2005,31083,1189,66,biblegateway


In [32]:
joined_metadata_df

,path,language,title,year,number_verses,number_chapters,number_books,source
232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania
7,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Version,0,31102,1189,66,zefania
161,./../data/bibles/transformed_xml_with_year\SF_...,ARA,New Arabic Version (Ketab El Hayat),0,31101,1189,66,zefania
8,./../data/bibles/transformed_xml_with_year\SF_...,ARA,Smith Van Dyke Arabic Bible,0,31102,1189,66,zefania
162,./../data/bibles/transformed_xml_with_year\SF_...,AZB,Description_en=Southern Azeri Translation,0,31102,1189,66,zefania
...,...,...,...,...,...,...,...,...
101,/versions/Reina-Valera-Revisada-RVR/#booklist,SPA,Reina Valera Revisada (RVR1977),1977,31103,1189,66,biblegateway
102,/versions/Reina-Valera-1995-RVR1995-Biblia/#bo...,SPA,Reina-Valera 1995 (RVR1995),1995,31104,1189,66,biblegateway
103,/versions/Reina-Valera-Antigua-RVA-Biblia/#boo...,SPA,Reina-Valera Antigua (RVA),1569,31101,1189,66,biblegateway
104,/versions/Spanish-Blue-Red-Gold-Letter-Edition...,SPA,Spanish Blue Red and Gold Letter Edition (SRV-...,2014,31103,1189,66,biblegateway


In [33]:
joined_metadata_df.reset_index(inplace=True)

In [34]:
joined_metadata_df

,index,path,language,title,year,number_verses,number_chapters,number_books,source
0,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania
1,7,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Version,0,31102,1189,66,zefania
2,161,./../data/bibles/transformed_xml_with_year\SF_...,ARA,New Arabic Version (Ketab El Hayat),0,31101,1189,66,zefania
3,8,./../data/bibles/transformed_xml_with_year\SF_...,ARA,Smith Van Dyke Arabic Bible,0,31102,1189,66,zefania
4,162,./../data/bibles/transformed_xml_with_year\SF_...,AZB,Description_en=Southern Azeri Translation,0,31102,1189,66,zefania
...,...,...,...,...,...,...,...,...,...
216,101,/versions/Reina-Valera-Revisada-RVR/#booklist,SPA,Reina Valera Revisada (RVR1977),1977,31103,1189,66,biblegateway
217,102,/versions/Reina-Valera-1995-RVR1995-Biblia/#bo...,SPA,Reina-Valera 1995 (RVR1995),1995,31104,1189,66,biblegateway
218,103,/versions/Reina-Valera-Antigua-RVA-Biblia/#boo...,SPA,Reina-Valera Antigua (RVA),1569,31101,1189,66,biblegateway
219,104,/versions/Spanish-Blue-Red-Gold-Letter-Edition...,SPA,Spanish Blue Red and Gold Letter Edition (SRV-...,2014,31103,1189,66,biblegateway


In [35]:
joined_metadata_df.loc[joined_metadata_df["source"] == "zefania", "file_name"] =  joined_metadata_df.path.str.slice(33,-4)

In [36]:
joined_metadata_df.loc[joined_metadata_df["source"] == "biblegateway", "file_name"] =  joined_metadata_df.title

In [37]:
joined_metadata_df

,index,path,language,title,year,number_verses,number_chapters,number_books,source,file_name
0,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,with_year\SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBA...
1,7,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Version,0,31102,1189,66,zefania,with_year\SF_2009-01-20_ALB_ALB1_(ALBANIAN VER...
2,161,./../data/bibles/transformed_xml_with_year\SF_...,ARA,New Arabic Version (Ketab El Hayat),0,31101,1189,66,zefania,with_year\SF_2012-01-07_ARA_ARANAV_(NEW ARABIC...
3,8,./../data/bibles/transformed_xml_with_year\SF_...,ARA,Smith Van Dyke Arabic Bible,0,31102,1189,66,zefania,with_year\SF_2009-01-20_ARA_ARASVD_(SMITH VAN ...
4,162,./../data/bibles/transformed_xml_with_year\SF_...,AZB,Description_en=Southern Azeri Translation,0,31102,1189,66,zefania,with_year\SF_2012-01-07_AZB_AZERI_(DESCRIPTION...
...,...,...,...,...,...,...,...,...,...,...
216,101,/versions/Reina-Valera-Revisada-RVR/#booklist,SPA,Reina Valera Revisada (RVR1977),1977,31103,1189,66,biblegateway,Reina Valera Revisada (RVR1977)
217,102,/versions/Reina-Valera-1995-RVR1995-Biblia/#bo...,SPA,Reina-Valera 1995 (RVR1995),1995,31104,1189,66,biblegateway,Reina-Valera 1995 (RVR1995)
218,103,/versions/Reina-Valera-Antigua-RVA-Biblia/#boo...,SPA,Reina-Valera Antigua (RVA),1569,31101,1189,66,biblegateway,Reina-Valera Antigua (RVA)
219,104,/versions/Spanish-Blue-Red-Gold-Letter-Edition...,SPA,Spanish Blue Red and Gold Letter Edition (SRV-...,2014,31103,1189,66,biblegateway,Spanish Blue Red and Gold Letter Edition (SRV-...


In [38]:
joined_metadata_df["file_name"] = joined_metadata_df["file_name"].str.replace(r"^.+?\\", "")


In [39]:
joined_metadata_df.loc[joined_metadata_df["file_name"].str.contains(r".*?reina.*?")]

,index,path,language,title,year,number_verses,number_chapters,number_books,source,file_name


In [40]:
joined_metadata_df.to_csv("./../data/metadata_translations.tsv", sep="\t")

In [41]:
joined_metadata_df

,index,path,language,title,year,number_verses,number_chapters,number_books,source,file_name
0,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...
1,7,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Version,0,31102,1189,66,zefania,SF_2009-01-20_ALB_ALB1_(ALBANIAN VERSION)
2,161,./../data/bibles/transformed_xml_with_year\SF_...,ARA,New Arabic Version (Ketab El Hayat),0,31101,1189,66,zefania,SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (...
3,8,./../data/bibles/transformed_xml_with_year\SF_...,ARA,Smith Van Dyke Arabic Bible,0,31102,1189,66,zefania,SF_2009-01-20_ARA_ARASVD_(SMITH VAN DYKE ARABI...
4,162,./../data/bibles/transformed_xml_with_year\SF_...,AZB,Description_en=Southern Azeri Translation,0,31102,1189,66,zefania,SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHE...
...,...,...,...,...,...,...,...,...,...,...
216,101,/versions/Reina-Valera-Revisada-RVR/#booklist,SPA,Reina Valera Revisada (RVR1977),1977,31103,1189,66,biblegateway,Reina Valera Revisada (RVR1977)
217,102,/versions/Reina-Valera-1995-RVR1995-Biblia/#bo...,SPA,Reina-Valera 1995 (RVR1995),1995,31104,1189,66,biblegateway,Reina-Valera 1995 (RVR1995)
218,103,/versions/Reina-Valera-Antigua-RVA-Biblia/#boo...,SPA,Reina-Valera Antigua (RVA),1569,31101,1189,66,biblegateway,Reina-Valera Antigua (RVA)
219,104,/versions/Spanish-Blue-Red-Gold-Letter-Edition...,SPA,Spanish Blue Red and Gold Letter Edition (SRV-...,2014,31103,1189,66,biblegateway,Spanish Blue Red and Gold Letter Edition (SRV-...


In [42]:
joined_metadata_df.language.unique()

array(['ALB', 'ARA', 'AZB', 'BAQ', 'BEL', 'BLA', 'BUL', 'CEB', 'CHA',
       'CHI', 'CKB', 'COP', 'CZE', 'DAN', 'ENG', 'ESP', 'FIN', 'FRE',
       'GAE', 'GER', 'GOT', 'GRC', 'HAT', 'HEB', 'HIN', 'HUN', 'ITA',
       'JAM', 'KOR', 'LAT', 'MAR', 'NDS', 'NL_', 'NOR', 'PON', 'POR',
       'RUM', 'RUS', 'SCR', 'SHU', 'SPA', 'SWA', 'SWE', 'SYR', 'TAM',
       'TGL', 'THA', 'TUR', 'UND', 'VIE', 'WIU', 'XKL', 'YBY', 'ZIA'],
      dtype=object)